In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service


from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from webdriver_manager.microsoft import EdgeChromiumDriverManager



from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select




from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException, StaleElementReferenceException, NoSuchElementException
import time
import traceback

In [4]:
import pandas as pd
from datetime import datetime


pos = 2
autentica = pd.read_excel('ls.xlsx')
login = str(autentica['login'][pos])

autentica['senha'] = autentica['senha'].astype(str)
autentica['senha'] = autentica['senha'].str.zfill(11)

senha = autentica['senha'][pos]

display(autentica)
name = str(autentica['ubs'][pos])
data_e_hora_atuais = datetime.now()
data_e_hora = data_e_hora_atuais.strftime('%Y-%m-%d_%H-%M')

raceList = ['BRANCA', 'PRETA', 'AMARELA', 'PARDA', 'INDÍGENA', 'SEM INFORMAÇÃO']


# nav = webdriver.Chrome(ChromeDriverManager().install())
# nav = webdriver.Edge(EdgeChromiumDriverManager().install())


,login,senha,ubs
0,joana,05216791418,borges
1,agda,09947741419,dois_riachos
2,amanda,05067676477,sobreira


In [5]:
def openEpHealph():
    
#     nav = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
#     nav = webdriver.Chrome(ChromeDriverManager().install())
    global nav
    nav = webdriver.Firefox(executable_path=GeckoDriverManager().install())
    nav.get("https://afogadosdaingazeira.ephealth.com.br/portal/login")
    
    # inserindo usuário
    nav.find_element(By.ID, 'EMail').click()
    nav.find_element(By.ID, 'EMail').send_keys(login)

    # inserindo senha 
    nav.find_element(By.ID, 'Password').click()
    nav.find_element(By.ID, 'Password').send_keys(senha)

    nav.find_element(By.XPATH, '/html/body/form/button').click()

In [6]:
def people(has_dm):
    
    openEpHealph()
        
    print(f'Extração iniciada as: {datetime.now()}')

    nav.get("https://afogadosdaingazeira.ephealth.com.br/portal/individuo")
    
    base = nav.find_element(By.XPATH, has_dm).click()

#     children = nav.find_element(By.ID, 'maxAge')
#     nav.execute_script("arguments[0].setAttribute('value', '4.0')",children)
#     time.sleep(1)
    
#     hypertension = nav.find_element(By.XPATH, '//*[@id="img_hipertensaoFlag"]').click()
#     time.sleep(1)

#     diabetes = nav.find_element(By.XPATH, '//*[@id="img_diabetesFlag"]').click()
#     time.sleep(1)

#     pregnant = nav.find_element(By.XPATH, '//*[@id="img_gestanteFlag"]').click()
#     time.sleep(1)

    """
    race = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/form/div[3]/div[5]/span/span[1]/span/ul/li/input')
    race.click()
    time.sleep(1)
    race.send_keys(raceList[race_item])
    race.send_keys(Keys.ENTER)
    """

    time.sleep(1)

    unity = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/form/div[4]/div[1]/span/span[1]/span/span[2]')
    unity.click()
    time.sleep(1)
    nav.find_element(By.XPATH,'/html/body/span/span/span[1]/input').send_keys(Keys.ENTER)
    time.sleep(1)
    nav.find_element(By.ID, 'button').click()

    time.sleep(1)
    nav.find_element(By.ID, 'card-toggle').click()
    time.sleep(10)

In [7]:
def get_table(has_dm):
    
    list_links = []
    people(has_dm)
    
    print(f'Iniciando lista de links: {datetime.now()}')

    exit = False
    
    peoples = nav.find_element(By.CLASS_NAME, 'dataTables_scrollBody')
    link = peoples.find_elements(By.TAG_NAME, 'a') 
    
    
    
    while exit is False:
        try:
            for a in link:
                
                list_links.append(a.get_attribute('href'))
                
            page = nav.find_element(By.CSS_SELECTOR, f"#individuo-list-datatable_next > a:nth-child(1)")

            page.click()
            time.sleep(10)                
            
        except (ElementClickInterceptedException, NoSuchElementException):
#             if ElementClickInterceptedException:    
#                 traceback.print_exc()
#             if NoSuchElementException: 
#                 traceback.print_exc()
            print('acabaram as paginas')
            break
        
        except StaleElementReferenceException:
            peoples = nav.find_element(By.CLASS_NAME, 'dataTables_scrollBody')
            link = peoples.find_elements(By.TAG_NAME, 'a') 

    
    list_links = list(dict.fromkeys(list_links))
    print(f'lista de links finalizada as: {datetime.now()}')
    
    return list_links    


In [8]:
def get_individual_info(has_dm, p_has, p_dm):
    
#   name sex b_date cpf cns has dm med_rec race m_name, proffisional, unity
    name, sex, b_date, cpf, cns, has, dm, med_rec, race, m_name, profissional, unity = [], [], [], [], [], [], [], [], [], [], [], []

    
    individual = get_table(has_dm)
    
    print(f'Iniciando individuos : {datetime.now()}')
    for i in individual:
        
        time.sleep(1)
        
        nav.get(i)
        
        i_name = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/h5').text
        name.append(i_name)
        
        i_sex = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/p[1]').text
        i_sex = i_sex.split(',')
        sex.append(i_sex[0].upper())
        
        i_bdate = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[3]/div/div/table/tbody/tr[4]/td[2]').text
        i_bdate = i_bdate.split(";")
        b_date.append(i_bdate[0])
        
        linha_dois = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[2]/td[1]').text
        linha_tres = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[3]/td[1]').text
        
        linha_dois_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[2]/td[2]').text
        linha_tres_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[3]/td[2]').text
        
        if(linha_dois.upper() == "CPF"):
            cpf.append(linha_dois_text)
            
        elif(linha_tres.upper() == "CPF"):
            cpf.append(linha_tres_text)
            
        else:
            cpf.append("--")
        
        i_cns = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/p[2]').text
        i_cns = i_cns.split(':')
        if i_cns[0].upper() != 'CNS':
            cns.append('--')
        else:
            cns.append(i_cns[1])
        
        i_has = p_has
        has.append(i_has)
        
        i_dm = p_dm
        dm.append(i_dm)
        
        i_medrec = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[3]/div/div/table/tbody/tr[1]/td[2]').text
        med_rec.append(i_medrec)
        
        i_race = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[3]/div/div/table/tbody/tr[5]/td[2]').text
        race.append(i_race.upper())
        
        i_mname = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[3]/div/div/table/tbody/tr[6]/td[2]').text
        m_name.append(i_mname)
    
        try:
            i_uni = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/p[3]').text
        
        except NoSuchElementException:
            i_uni = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/p[2]').text
        
        i_uni = i_uni.split(':')
        unity.append(i_uni[1])
        
        try:
            linha_tres = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[3]/td[1]').text
            linha_tres_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[3]/td[2]').text
            if(linha_tres.upper() == 'PROFISSIONAL'):
                profissional.append(linha_tres_text)
        except:
            continue
        try:
            linha_quatro = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[4]/td[1]').text
            linha_quatro_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[4]/td[2]').text
            if(linha_quatro.upper() == 'PROFISSIONAL'):
                profissional.append(linha_quatro_text)
        except:
            continue
        
        try:
            linha_cinco = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[5]/td[1]').text
            linha_cinco_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[5]/td[2]').text
            if(linha_cinco.upper() == 'PROFISSIONAL'):
                profissional.append(linha_cinco_text)
        except:
            continue
        
        try:
            linha_seis = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[6]/td[1]').text
            linha_seis_text = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[6]/td[2]').text
            if(linha_seis.upper() == 'PROFISSIONAL'):
                profissional.append(linha_seis_text)
            
        except:
            continue
            
        time.sleep(1)
        
    print(f'Finalizando individuos : {datetime.now()}')

    dict_individual = {"name": name, "sex":sex, "b_date":b_date, "cpf":cpf, "cns":cns, "has":has, "dm":dm, "med_rec":med_rec, "race":race, "m_name":m_name, "profissional":profissional, "unity":unity, "link":individual}
    df_individual = pd.DataFrame(data=dict_individual)    
    
    print(f'Extração finalizada as: {datetime.now()}')   
    
    nav.quit()
    
    return df_individual

# indi = get_indivivual_info()

In [9]:
def get_has_dm():
    has_dm_click = ['//*[@id="img_hipertensaoFlag"]', '//*[@id="img_diabetesFlag"]']
    has = get_individual_info(has_dm_click[0], 'sim', 'nao')
    dm = get_individual_info(has_dm_click[1], 'nao', 'sim')
    
    has_dm = pd.concat([has, dm], ignore_index=True)
    
    double = has_dm[has_dm.duplicated(subset=['name', 'cpf'])]
    double.index = [x for x in range(0, len(double))]
    double['has'] = double['has'].replace(['nao'], 'sim')
    double['dm'] = double['dm'].replace(['nao'], 'sim')
    
    final = pd.concat([has_dm, double], ignore_index=True)
    final = final.drop_duplicates(subset=['name', 'cpf'], keep='last')
    final = final.sort_values(by=['name'])
    final.index = [x for x in range(0, len(final))]
    
    return final
    

In [10]:
unification = get_has_dm()

[WDM] - Downloading: 19.9kB [00:00, 4.57MB/s]                   
[WDM] - Downloading: 19.9kB [00:00, 8.28MB/s]                   
[WDM] - Downloading: 100%|██████████| 3.10M/3.10M [00:00<00:00, 12.9MB/s]


Extração iniciada as: 2024-02-06 14:18:01.534688
Iniciando lista de links: 2024-02-06 14:18:17.616963
acabaram as paginas
lista de links finalizada as: 2024-02-06 14:20:15.866751
Iniciando individuos : 2024-02-06 14:20:15.866779
Finalizando individuos : 2024-02-06 15:00:21.384510
Extração finalizada as: 2024-02-06 15:00:21.387833
Extração iniciada as: 2024-02-06 15:00:28.440442
Iniciando lista de links: 2024-02-06 15:00:44.397112
acabaram as paginas
lista de links finalizada as: 2024-02-06 15:01:16.998601
Iniciando individuos : 2024-02-06 15:01:16.998618
Finalizando individuos : 2024-02-06 15:13:03.457042
Extração finalizada as: 2024-02-06 15:13:03.459501


/tmp/ipykernel_15344/3913768521.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  double['has'] = double['has'].replace(['nao'], 'sim')
/tmp/ipykernel_15344/3913768521.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  double['dm'] = double['dm'].replace(['nao'], 'sim')


In [11]:
display(unification)
unification = unification[['name', 'sex', 'b_date', 'cpf', 'cns', 'has', 'dm', 'med_rec', 'race', 'm_name', 'link']]
unification.to_excel(f'{data_e_hora}_{name}_has_dm.xlsx', index=False)

,name,sex,b_date,cpf,cns,has,dm,med_rec,race,m_name,profissional,unity,link
0,ABDIAS JACINTO DA SILVA,MASCULINO,22/04/1937,31713890100,705002214245559,sim,nao,--,PARDA,MARIA DA CONCEIÇÃO,HELENA ALVES DA SILVA ALMEIDA,PSF SOBREIRA,https://afogadosdaingazeira.ephealth.com.br/po...
1,ACEMIRA DA SILVA ALVES,FEMININO,03/03/1945,37042793404,700004537413802,sim,nao,--,PARDA,ELISA ALVES DA SILVA,ELAINE MACHADO EVANGELISTA,PSF SOBREIRA,https://afogadosdaingazeira.ephealth.com.br/po...
2,ACILDA DE GÓES SILVA,FEMININO,14/02/1969,54799848453,708202634628143,sim,nao,--,BRANCA,EROTIDES MARIA DE GÓES SILVA,TERESINHA RUFINO DA SILVA,PSF SOBREIRA,https://afogadosdaingazeira.ephealth.com.br/po...
3,ADAILTON LAURINDO DA SILVA,MASCULINO,18/09/1953,07874222407,700401371184950,sim,nao,01565,PARDA,SEVERINA MARIA DA CONCEICAO,MARIA DA PAZ OLIVEIRA SILVA,PSF SOBREIRA,https://afogadosdaingazeira.ephealth.com.br/po...
4,ADALBERTO BEZERRA DA SILVA FILHO,MASCULINO,23/01/1986,33917528827,709205269812939,sim,nao,--,BRANCA,MARIA DO SOCORRO FLORENTINO BEZERRA,ROGERIO JESUINO DE OLIVEIRA,PSF SOBREIRA,https://afogadosdaingazeira.ephealth.com.br/po...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
640,VILACIA MARIA DE ALCANTARA,FEMININO,18/09/1945,79509410497,700103904395215,sim,nao,--,BRANCA,MARIA TEREZA DE LIMA,ROGERIO JESUINO DE OLIVEIRA,PSF SOBREIRA,https://afogadosdaingazeira.ephealth.com.br/po...
641,VILMA LUCIA PEREIRA SILVA,FEMININO,14/10/1973,03138916481,708005878472023,sim,nao,--,PARDA,MARIZA LEITE DA SILVA PEREIRA,HELENA ALVES DA SILVA ALMEIDA,PSF SOBREIRA,https://afogadosdaingazeira.ephealth.com.br/po...
642,VILMAR DO NASCIMENTO SILVA,MASCULINO,10/06/1976,86221477468,705106404588670,sim,nao,--,PARDA,TEREZINHA DO NASCIMENTO SILVA,ROGERIO JESUINO DE OLIVEIRA,PSF SOBREIRA,https://afogadosdaingazeira.ephealth.com.br/po...
643,VIRGÍNIA MARIA DA SILVA QUEIROZ,FEMININO,05/03/1972,86340930425,708402720302267,sim,nao,--,PARDA,CICERA MARIA DA SILVA QUEIROZ,FABIANA CRISTINA DA SILVA,PSF SOBREIRA,https://afogadosdaingazeira.ephealth.com.br/po...
